In [1]:
!pip install jieba
!pip install gensim

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 19.2 MB 50.7 MB/s eta 0:00:01
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314476 sha256=048dd1d76c83fdbb3cdceaa74d675d11d47fa06a7e3a902d703c333f77d5b88c
  Stored in directory: /home/ma-user/.cache/pip/wheels/86/bc/97/67c05f24b07573fd425039f944f8bc57c8dbc6f2c0bcc046fa
Successfully built jieba
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 24.0 MB 51.9 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 24.9 MB/s eta 0:00:01


In [2]:
import gzip
import tarfile

In [32]:
def un_gz(file_name):
    
    # 获取文件的名称，去掉后缀名
    f_name = file_name.replace(".gz", "")
    # 开始解压
    g_file = gzip.GzipFile(file_name)
    #读取解压后的文件，并写入去掉后缀名的同名文件（即得到解压后的文件）
    open(f_name, "wb+").write(g_file.read())
    g_file.close()
    
un_gz('tencent-ailab-embedding-zh-d100-v0.2.0-s.tar 1.gz')

In [2]:
def untar(fname, dirs):
  t = tarfile.open(fname)
  t.extractall(path = dirs)

untar("tencent-ailab-embedding.tar", ".")

In [3]:
import os
import math
from itertools import chain
# import gensim
import numpy as np
import mindspore
from mindspore.mindrecord import FileWriter
import jieba

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
train = pd.read_csv(r'data.txt', header=None, sep=',')
real_test = pd.read_csv(r'test.txt', header=None, sep='\t')
train.columns = ['label', 'sentence']
real_test.columns = ['sentence']

X_train_text, X_test_text, y_train, y_test = train_test_split(
    train['sentence'].apply(lambda w: ' '.join(jieba.cut(w))), train['label'], train_size=0.8, test_size=0.2, random_state=0)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.489 seconds.
Prefix dict has been built successfully.


In [5]:
X_train_text

9310     有点 坑爹 ， 点 了 三份 牛肉 只放 了 一份 ， 点 了 2 分 鹌鹑蛋 只放 了 一...
5421     两次 了 都 送错 粉 ， 下单 是 牛腩 粉 ， 送餐 的 单子 写 的 大 排粉 ， 实...
4549                                      有点 淡 ， 实在 量 太小 了
9610                                   我 的 发票 呢 ？ 为什么 没有 啊
5579                                             等 了 70 分钟
                               ...                        
4859                  外卖 员 太 差劲 了 。 , 速度慢 ， , 豆浆 还 没 给 我 。
3264                                          这次 不错 ， 赞 一个
9845                                   味道 很 一般 ， 比老车 记 差远了
10799                                     单子 打错 了 ， 少送 了 。
2732                                 快递 师傅 态度 超好 ， 速度 一流 。
Name: sentence, Length: 8789, dtype: object

In [6]:
def read_imdb(path, seg='train'):
    labels = ['pos', 'neg']
    data = []
    for label in labels:
        files = os.listdir(os.path.join(path, seg, label))
        for file in files:
            with open(os.path.join(path, seg, label, file), 'r', encoding='utf8') as rf:
                review = rf.read().replace('\n', '')
                if label == 'pos':
                    data.append([review, 1])
                elif label == 'neg':
                    data.append([review, 0])
    return data

def tokenize_samples(raw_data):
    tokenized_data = []
    for review in raw_data:
        tokenized_data.append([tok.lower() for tok in review.split()])
    return tokenized_data

def encode_samples(tokenized_samples, word_to_idx):
    """
    tokenized_samples: [[word, word, ...]]
    word_to_idx: {word:idx, word:idx, ...}
    features: [[idx, idx, ...], [idx, idx, ...], ...]
    """
    features = []
    for sample in tokenized_samples:
        feature = []
        for token in sample:
            feature.append(word_to_idx.get(token, 0))
        features.append(feature)
    return features

def pad_samples(features, maxlen=500, pad=0):
    padded_features = []
    for feature in features:
        if len(feature) >= maxlen:
            padded_feature = feature[:maxlen]
        else:
            padded_feature = feature
            while len(padded_feature) < maxlen:
                padded_feature.append(pad)
        padded_features.append(padded_feature)
    return padded_features

def prepare_data(imdb_data_path='./data/imdb/aclImdb'):      
    # raw_data_train = read_imdb(imdb_data_path, seg='train')
    # raw_data_test = read_imdb(imdb_data_path, seg='test')
    # y_train = np.array([label for _, label in raw_data_train]).astype(np.int32)
    # y_test = np.array([label for _, label in raw_data_test]).astype(np.int32)    
    # tokenized_data_train = tokenize_samples([review for review, _ in raw_data_train])
    # tokenized_data_test = tokenize_samples([review for review, _ in raw_data_test])
    
    tokenized_data_train = tokenize_samples(X_train_text)
    tokenized_data_test = tokenize_samples(X_test_text)    
    vocab = set(chain(*tokenized_data_train))
    word_to_idx = {word: i+1 for i, word in enumerate(vocab)}
    word_to_idx['<unk>'] = 0
    X_train = np.array(pad_samples(encode_samples(tokenized_data_train, word_to_idx))).astype(np.int32)
    X_test = np.array(pad_samples(encode_samples(tokenized_data_test, word_to_idx))).astype(np.int32)
    return X_train, y_train, X_test, y_test, word_to_idx

X_train, y_train, X_test, y_test, word_to_idx = prepare_data()

In [8]:
X_real_test_text=real_test['sentence'].apply(lambda w: ' '.join(jieba.cut(w)))
tokenized_data_real_test = tokenize_samples(X_real_test_text)
X_real_test=np.array(pad_samples(encode_samples(tokenized_data_real_test, word_to_idx))).astype(np.int32)
# y_real_test=train['label']

In [7]:
# tokenized_data_train = tokenize_samples(X_train_text)
# tokenized_data_train
# word_to_idx
import gensim

In [8]:
X_train[7,:]

array([2137, 1905, 5624, 5144, 4462, 1704, 3455, 1712, 3420, 7034, 3699,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [11]:
pwd

'/home/ma-user/work'

In [13]:
# glove_file_path=r'./embedding/tencent-ailab-embedding.txt'
# word2vector=gensim.models.KeyedVectors.load_word2vec_format(glove_file_path,binary=False)
# word2vector.vocab

In [15]:
#!sed -i '1i\400000 50' ./data/glove/glove.6B.50d.txt
def load_embeddings(glove_file_path, word_to_idx, embed_size=100):
    word2vector = gensim.models.KeyedVectors.load_word2vec_format(
        glove_file_path, binary=False, encoding='utf-8')
    assert embed_size == word2vector.vector_size
    embeddings = np.zeros((len(word_to_idx), embed_size)).astype(np.float32)
    for word, idx in word_to_idx.items():
        try:
            embeddings[idx, :] = word2vector.word_vec(word)
        except KeyError:
            continue
    return embeddings
# embeddings = load_embeddings('./data/glove/glove.6B.50d.txt', word_to_idx)
# embeddings = load_embeddings(glove_file_path, word_to_idx)

In [21]:
# np.savetxt("./embedding/weight.txt", embeddings)

In [17]:
def get_json_data_list(X, y):
    data_list = []
    for i, (feature, label) in enumerate(zip(X, y)):
        data_json = {"id": i, "feature": feature.reshape(-1), "label": int(label)}
        data_list.append(data_json)
    return data_list

def convert_np_to_mindrecord(X_train, y_train, X_test, y_test, mindrecord_save_path="./mindrecord"):
    schema_json = {"id": {"type": "int32"},
                  "label": {"type": "int32"},
                  "feature": {"type": "int32", "shape": [-1]}}
    writer = FileWriter(os.path.join(mindrecord_save_path, "aclImdb_train.mindrecord"), shard_num=4)
    data_train = get_json_data_list(X_train, y_train)
    writer.add_schema(schema_json, "nlp_schema")
    writer.add_index(["id", "label"])
    writer.write_raw_data(data_train)
    writer.commit()
    
    writer = FileWriter(os.path.join(mindrecord_save_path, "aclImdb_test.mindrecord"), shard_num=4)
    data_test = get_json_data_list(X_test, y_test)
    writer.add_schema(schema_json, "nlp_schema")
    writer.add_index(["id", "label"])
    writer.write_raw_data(data_test)
    writer.commit()

In [10]:
X_train.max()+1

9375

In [11]:
# convert_np_to_mindrecord(X_train, y_train, X_test, y_test)

In [15]:
# np.zeros(X_real_test.shape[0])
# y_test
# np.array([0]*)

In [18]:
convert_np_to_mindrecord(X_real_test,np.zeros(X_real_test.shape[0]),X_real_test, np.zeros(X_real_test.shape[0]),
                        mindrecord_save_path="./mindrecordtest")

In [12]:
y_train.shape

(8789,)

In [31]:
import mindspore.dataset as mds
def create_dataset(base_path, batch_size, num_epochs, is_train):
    columns_list = ["feature", "label"]
    num_consumer = 4
    if is_train:
        path = os.path.join(base_path, "aclImdb_train.mindrecord0")
    else:
        path = os.path.join(base_path, "aclImdb_test.mindrecord0")
    dataset = mds.MindDataset(path, columns_list=["feature", "label"], num_parallel_workers=4)
    dataset = dataset.shuffle(buffer_size=dataset.get_dataset_size())
    dataset = dataset.batch(batch_size=batch_size, drop_remainder=True)
    dataset = dataset.repeat(count=num_epochs)
    return dataset
dataset_train = create_dataset("./mindrecord", batch_size=32, num_epochs=10, is_train=True)

In [32]:
# Copyright 2020 Huawei Technologies Co., Ltd
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ============================================================================
"""LSTM."""
import math

import numpy as np

from mindspore import Tensor, nn, context, Parameter, ParameterTuple
from mindspore.common.initializer import initializer
from mindspore.ops import operations as P

STACK_LSTM_DEVICE = ["CPU"]


# Initialize short-term memory (h) and long-term memory (c) to 0
def lstm_default_state(batch_size, hidden_size, num_layers, bidirectional):
    """init default input."""
    num_directions = 2 if bidirectional else 1
    h = Tensor(np.zeros((num_layers * num_directions, batch_size, hidden_size)).astype(np.float32))
    c = Tensor(np.zeros((num_layers * num_directions, batch_size, hidden_size)).astype(np.float32))
    return h, c


def stack_lstm_default_state(batch_size, hidden_size, num_layers, bidirectional):
    """init default input."""
    num_directions = 2 if bidirectional else 1

    h_list = c_list = []
    for _ in range(num_layers):
        h_list.append(Tensor(np.zeros((num_directions, batch_size, hidden_size)).astype(np.float32)))
        c_list.append(Tensor(np.zeros((num_directions, batch_size, hidden_size)).astype(np.float32)))
    h, c = tuple(h_list), tuple(c_list)
    return h, c


class StackLSTM(nn.Cell):
    """
    Stack multi-layers LSTM together.
    """

    def __init__(self,
                 input_size,
                 hidden_size,
                 num_layers=1,
                 has_bias=True,
                 batch_first=False,
                 dropout=0.0,
                 bidirectional=False):
        super(StackLSTM, self).__init__()
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.transpose = P.Transpose()

        # direction number
        num_directions = 2 if bidirectional else 1

        # input_size list
        input_size_list = [input_size]
        for i in range(num_layers - 1):
            input_size_list.append(hidden_size * num_directions)

        # layers
        layers = []
        for i in range(num_layers):
            layers.append(nn.LSTM(input_size=input_size_list[i],
                                      hidden_size=hidden_size,
                                      has_bias=has_bias,
                                      batch_first=batch_first,
                                      bidirectional=bidirectional,
                                      dropout=dropout))

        # weights
        weights = []
        for i in range(num_layers):
            # weight size
            weight_size = (input_size_list[i] + hidden_size) * num_directions * hidden_size * 4
            if has_bias:
                bias_size = num_directions * hidden_size * 4
                weight_size = weight_size + bias_size

            # numpy weight
            stdv = 1 / math.sqrt(hidden_size)
            w_np = np.random.uniform(-stdv, stdv, (weight_size, 1, 1)).astype(np.float32)

            # lstm weight
            weights.append(Parameter(initializer(Tensor(w_np), w_np.shape), name="weight" + str(i)))

        #
        self.lstms = layers
        self.weight = ParameterTuple(tuple(weights))

    def construct(self, x, hx):
        """construct"""
        if self.batch_first:
            x = self.transpose(x, (1, 0, 2))
        # stack lstm
        h, c = hx
        hn = cn = None
        for i in range(self.num_layers):
            x, hn, cn, _, _ = self.lstms[i](x, h[i], c[i], self.weight[i])
        if self.batch_first:
            x = self.transpose(x, (1, 0, 2))
        return x, (hn, cn)


class SentimentNet(nn.Cell):
    """Sentiment network structure."""

    def __init__(self,
                 vocab_size,
                 embed_size,
                 num_hiddens,
                 num_layers,
                 bidirectional,
                 num_classes,
                 weight,
                 batch_size):
        super(SentimentNet, self).__init__()
        # Mapp words to vectors
        self.embedding = nn.Embedding(vocab_size,
                                      embed_size,
                                      embedding_table=weight)
        self.embedding.embedding_table.requires_grad = False
        self.trans = P.Transpose()
        self.perm = (1, 0, 2)

        if context.get_context("device_target") in STACK_LSTM_DEVICE:
            # stack lstm by user
            self.encoder = StackLSTM(input_size=embed_size,
                                     hidden_size=num_hiddens,
                                     num_layers=num_layers,
                                     has_bias=True,
                                     bidirectional=bidirectional,
                                     dropout=0.0)
            self.h, self.c = stack_lstm_default_state(batch_size, num_hiddens, num_layers, bidirectional)
        else:
            # standard lstm
            self.encoder = nn.LSTM(input_size=embed_size,
                                   hidden_size=num_hiddens,
                                   num_layers=num_layers,
                                   has_bias=True,
                                   bidirectional=bidirectional,
                                   dropout=0.0)
            self.h, self.c = lstm_default_state(batch_size, num_hiddens, num_layers, bidirectional)

        self.concat = P.Concat(1)
        if bidirectional:
            self.decoder = nn.Dense(num_hiddens * 4, num_classes)
        else:
            self.decoder = nn.Dense(num_hiddens * 2, num_classes)

    def construct(self, inputs):
        # input：(64,500,300)
        embeddings = self.embedding(inputs)
        embeddings = self.trans(embeddings, self.perm)
        output, _ = self.encoder(embeddings, (self.h, self.c))
        # states[i] size(64,200)  -> encoding.size(64,400)
        encoding = self.concat((output[0], output[499]))
        outputs = self.decoder(encoding)
        return outputs

In [15]:
embedding_tabel = np.loadtxt(os.path.join("./embedding/", "weight.txt")).astype(np.float32)

In [33]:
from mindspore import Tensor, nn, Model, context, Parameter
from mindspore.common.initializer import initializer
from mindspore.ops import operations as P
from mindspore.nn import Accuracy
from mindspore.train.callback import LossMonitor, CheckpointConfig, ModelCheckpoint, TimeMonitor
# from mindspore.model_zoo.lstm import SentimentNet

network = SentimentNet(vocab_size=embedding_tabel.shape[0],
                embed_size=100,
                num_hiddens=100,
                num_layers=2,
                bidirectional=False,
                num_classes=2,
                weight=Tensor(embedding_tabel),
                batch_size=32)

loss = nn.SoftmaxCrossEntropyWithLogits( sparse=True)
opt = nn.Momentum(network.trainable_params(), 0.1, 0.9)
loss_callback = LossMonitor(per_print_times=60)
model = Model(network, loss, opt, {'acc': Accuracy()})
config_ck = CheckpointConfig(save_checkpoint_steps=390, keep_checkpoint_max=10)
checkpoint_cb = ModelCheckpoint(prefix="lstm", directory="./model", config=config_ck)

from mindspore import context
context.set_context(mode=context.GRAPH_MODE, save_graphs=False, device_target="CPU")
model.train(1, dataset_train, callbacks=[checkpoint_cb, loss_callback], dataset_sink_mode=False)

[CRITICAL] PARSER(2095,ffff9c91c780,python):2023-01-15-20:49:15.164.891 [mindspore/ccsrc/pipeline/jit/parse/resolve.cc:158] ResolveParameterObj] The parameter construct_wrapper.689:weight_ih_l0 , its name 'weight_ih_l0' already exists. Please set a unique name for the parameter.


RuntimeError: mindspore/ccsrc/pipeline/jit/parse/resolve.cc:158 ResolveParameterObj] The parameter construct_wrapper.689:weight_ih_l0 , its name 'weight_ih_l0' already exists. Please set a unique name for the parameter.

# In file /home/ma-user/anaconda3/envs/MindSpore/lib/python3.7/site-packages/mindspore/nn/layer/rnns.py(460)
        for i in range(self.num_layers):
        ^
